In [1]:
import numpy as np
import plotly
import plotly.graph_objs as go
import torch
from torch.utils.data import DataLoader
from experiment import Net, InitDataset, CreateModel, LoadCheckpoint, RunOne
from yaml import safe_load
import torch.nn as nn

In [2]:
#good chars: dfb53a6b2
#perfect emb: a2bb6c3c8

In [3]:
exphash = 'aa0d32e62'
exp = 'results/%s' % exphash
with open(exp) as f:
  h = safe_load(f.read())
h = h['exp_info']  

In [40]:
h['add_random_count'] = 0
h['space_frequency']=0
h['add_next_words']=4

In [41]:
ds = InitDataset(h)

In [42]:
#h['hyperparameters']['manual_attention'] = True
model = CreateModel(h['hyperparameters'])
LoadCheckpoint(model, 'models/%s'%exphash)

model = model.cuda()


[14:50:57] loading configuration file bert_distil_uncased/config.json


[14:50:57] loading weights file bert_distil_uncased/pytorch_model.bin


FileNotFoundError: [Errno 2] No such file or directory: 'models/aa0d32e62'

In [43]:
h['dataset_split'][1] = .01

In [44]:
val_loader = DataLoader(ds[1], batch_size=64, shuffle=False)

In [45]:
ds[1][0]

(tensor([ 4, 11, 46, 47]),)


{'label_ids': tensor([ 2000,  3058,  2045,  2003,  2053,  2338,  1997, 15111, 10505, 11262,
          2626,  1037,  2048,  5717,  2337,  2028,  3157,  2698,  2416,  3661,
          2000,  2720,  1996,  2613, 25323,  1997,  1996,  3259,  2003,  2008,
          2017,  2064,  1056,  2275,  1996,  2338,  1997, 15111, 10505,  2091,
          5973,  2138,  2009,  2003,  7214,  1998,  2097,  2196,  3113,  1996,
          5918,  1997,  1996,  6900,  7905,  8780,  6483, 14387,  2096,  2169,
          1997,  1996, 17289,  9339,  4216,  2024,  4089,  2310,  3089, 22749,
          3468,  4401,  2228,  2009,  5875,  2000,  3602,  2008,  1996, 17627,
          2277,  3075,  2515,  5383,  4809,  1997,  2169,  2021,  2027,  2024,
          2025,  2112,  1997,  2151, 17627,  8882]),
 'token_ids': tensor([ 2000,  3058,  2045,  2003,   103,  2338,  1997, 15111, 10505, 11262,
          2626,   103,  2048,  5717,  2337,  2028,  3157,  2698,  2416,  3661,
          2000,  2720,  1996,  2613, 25323,  1997,  

In [46]:
char_to_idx_map = torch.load('../data/char_to_idx_map2.pt')

In [47]:
Torch2Py = lambda x: x.cpu().numpy().tolist()

In [48]:
idx_to_char_map= {v:k for k,v in char_to_idx_map.items()}
idx_to_tok_map = {}
with open('../data/bert-base-uncased-vocab.txt') as f:
  for i, l in enumerate(f.readlines()):
    idx_to_tok_map[i] = l.strip()
def GetChars(encoded):
  s = ""
  for c in encoded:
    s+= idx_to_char_map[c]
  return s.replace('\x00','_')
def GetToks(encoded):
  s = ""
  for c in encoded:
    s+= " " + idx_to_tok_map[c]
  return s

In [52]:
d = ds[1][9]

(tensor([ 1, 14, 19, 25, 28, 43, 65, 67, 80, 85]),)


In [53]:
GetToks(Torch2Py(d['token_ids']))

' furthermore [MASK] movie titled awesome i fuck ##in shot that featuring a live performance [MASK] october nine two zero [MASK] four at madison square garden [MASK] scheduled for [MASK] on march three one two zero zero six directed by nathan ##ial h rn [MASK] ##ow r it was taped by five zero fans in the crowd who were given hi eight video cameras the film [MASK] first [MASK] on january six two zero zero six to the fans that shot [MASK] footage it premiered at [MASK] sundance film festival in january and the dvd of the'

In [54]:
for i in range(64):
  print(GetChars(Torch2Py(d['input_ids'][i])))

furthermore [m] mo
[m] movie titled a
movie titled aweso
titled awesome i i
awesome i fuckin s
i fuckin shot that
fuckin shot that f
in shot that featu
shot that featurin
that featuring a l
featuring a live p
a live performance
live performance [
performance [m] oc
[m] october nine t
october nine two z
nine two zero [m] 
two zero [m] four 
zero [m] four at m
[m] four at madiso
four at madison sq
at madison square 
madison square gar
square garden [m] 
garden [m] schedul
[m] scheduled for 
scheduled for [m] 
for [m] on march t
[m] on march three
on march three one
march three one tw
three one two zero
one two zero zero 
two zero zero six 
zero zero six dire
zero six directed 
six directed by ni
directed by nathai
by nathanial h rn_
nathanial h rn [m]
ial h rn [m]ow____
h rn [m]ow r______
rn [m]ow r it_____
[m]ow r it was____
ow r it was taped_
r it was taped by_
it was taped by fi
was taped by five 
taped by five zero
by five zero fans 
five zero fans in 
zero fans in the c
fans in the 

In [11]:
w = '[cls] guitars [m] some ot [m] [pad] [pad]'
thing = []
for c in w :
  thing.append(char_to_idx_map[c])
print(len(thing))
inp = torch.tensor(thing).unsqueeze(0)

41


In [12]:
inp

tensor([[32, 46, 55, 62, 33,  1, 50, 64, 52, 63, 44, 61, 62,  1, 32, 56, 33,  1,
         62, 58, 56, 48,  1, 58, 63,  1, 32, 56, 33,  1, 32, 59, 44, 47, 33,  1,
         32, 59, 44, 47, 33]])

In [13]:
inp_ids = ds[1][0]['input_ids'].unsqueeze(0)
label_ids = ds[1][0]['label_ids']

In [14]:
token_ids = ds[1][0]['token_ids'].unsqueeze(0)

In [15]:
for i in inp_ids[0]:
  print(GetChars(Torch2Py(i)))

[m]_______________
ing_______________
a_________________
quarter___________
is________________
the_______________
male______________
attacking_________
the_______________
female____________
and_______________
the_______________
remaining_________
[m]_______________
[m]_______________
females___________
attacking_________
their_____________
male______________
partner___________
[m]_______________
how_______________
many______________
instances_________
of________________
domestic__________
[m]_______________
actually__________
involve___________
male______________
victims___________
is________________
difficult_________
[m]_______________
domestic__________
violence__________
victims___________
may_______________
be________________
reluctant_________
to________________
get_______________
help______________
for_______________
a_________________
number____________
of________________
[m]_______________
see_______________
this______________
article___________
article___________
checked_____

In [16]:
GetToks(Torch2Py(label_ids))

' brawl ##ing a quarter is the male attacking the female and the remaining quarter being females attacking their male partner determining how many instances of domestic violence actually involve male victims is difficult male domestic violence victims may be reluctant to get help for a number of reasons see this article article checked august eight two zero zero four a man who calls for help may even risk being arrested as the per ##pet ##rator even though he was the victim the general consensus seems to be that male on female domestic violence is more likely'

In [17]:
inp_ids.shape #chars

torch.Size([1, 96, 18])

In [18]:
output = model(inp_ids.cuda())

In [19]:
moutwords = output.argmax(-1)[0]

In [20]:
# gives a garbage output
GetToks(Torch2Py(moutwords))

' . " a quarter is the male attacking the female and the female male male males attacking their male partner . how many instances of sexual violence actually multiple male victims are difficult romance domestic violence victims may be interested to please help please a number of please see from article article contents august eight two zero zero four a man and calls for help may , child victims arrested as male male pet ##rator even though he was the victim the general consensus seems . conclusion that assault on female domestic violence is more likely'

In [21]:
# we check the inputs that are put into  the distilbert model (assumption maybe wrong) ---hold on
inp_embs = model.bert.distilbert.inputs_embeds

In [22]:
# with and without position embeddings we for the most part get the word back checking this way.
inp_embs = model.embedded_chars
#inp_embs = model.embedded_chars_no_pos
word = (inp_embs@model.bert.distilbert.embeddings.word_embeddings.weight.data.T).argmax(-1)[0]
GetToks(Torch2Py(word))

' [CLS] ##ing a quarter is the male 910 the ##dna and the remaining [CLS] [CLS] ##dna ##rdon their male partner [CLS] how 690 ##rdon of domestic [CLS] 930 ##omba male victims is 670 [CLS] domestic ##omba victims may 243 ##hner to get help for a number of [CLS] see this article article checked 740 820 two zero zero 276 a 243 who calls for help may even [CLS] [CLS] ##anor as [CLS] [CLS] pet ##rator even though 670 was the ##anor the general consensus ##anor [CLS] [CLS] that [CLS] on ##dna domestic [CLS] 930 more likely'

In [23]:
# giving the model the token ids directly we get good results as expected. this is vanilla bert.
outputstok = model.bert(input_ids=token_ids)
tokoutwords = outputstok[0].argmax(-1)[0]
print(GetToks(Torch2Py(tokoutwords)))

# so if we run this and grab the input embeddings just before they go into the transformer
inp_emb_tok_pos = model.bert.distilbert.inputs_embeds
inp_emb_tok_no_pos = model.bert.distilbert.inputs_embeds_no_pos
outputstok = model.bert(inputs_embeds=inp_emb_tok_no_pos)
tokoutwords = outputstok[0].argmax(-1)[0]
print("AA")
print(GetToks(Torch2Py(tokoutwords)))
# no pos gives garbage


 one the one quarter is the male attacking the victim and the remaining quarter being female male the male partner . how many instances of domestic violence actually involve male victims is difficult male domestic violence victims may be reluctant to get help for a variety of reasons see this article article checked august eight two zero zero four a man who calls for help may even consider being identified as the per ##pet ##rator even though he was the victim the general consensus seems to be that focusing on female domestic violence is more likely
AA
 romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance romance 

In [24]:
# checking the result of looking at the input embeddings when compared directly with the emb matrix shows similar to the char predembs
out = model.bert.distilbert.inputs_embeds_no_pos
word = (out@model.bert.distilbert.embeddings.word_embeddings.weight.data.T).argmax(-1)[0]
GetToks(Torch2Py(word))

' brawl ##ing [CLS] quarter is the male ##rdon the [CLS] and [CLS] remaining quarter being females [CLS] [CLS] male partner [CLS] how many ##rdon of domestic ##anor 670 [CLS] male ##anor is 670 male domestic ##anor ##anor may be ##rdon to 930 help for a number of reasons see this article article ##rdon 740 840 two zero zero 276 a man who calls for help may even [CLS] being [CLS] as the per [CLS] ##rator even though 670 was the ##dna the general consensus ##anor to be that [CLS] on 690 domestic ##anor is [CLS] likely'

In [25]:
# if we input the embeddings from the char embedder with and without pos what happens?
# without pos its garbage just ------- across the board.
# with pos its basically the same but with a single . instead of all - - -

# BUG: adding the position embeddings from the vanilla bert model gives the right output. so im just adding the pos embeddings wrong in my experiments.
# i wonder if theyre getting initialized to new values when the model initializes?!
output = model(inp_ids.cuda())

inp_embs_no_pos = model.embedded_chars_no_pos
inp_embs = model.embedded_chars
outputsemb = model.bert(inputs_embeds=inp_embs+pos_emb)
charoutwords = outputsemb[0].argmax(-1)[0]
print(GetToks(Torch2Py(charoutwords)))
print()
charoutwords = output[0].argmax(-1)
print(GetToks(Torch2Py(charoutwords)))

NameError: name 'pos_emb' is not defined

In [38]:
# so the position embeddings are correct...
(model.pos_emb_inst == model.bert.distilbert.embeddings.position_embeddings_inst).float().mean()
# FAAAACK I FORGOT THE LAYER NORM

tensor(1., device='cuda:0')

tensor([[[ 1.7505e-02, -2.5631e-02, -3.6642e-02,  ...,  3.3437e-05,
           6.8312e-04,  1.5441e-02],
         [ 7.7580e-03,  2.2613e-03, -1.9444e-02,  ...,  2.8910e-02,
           2.9753e-02, -5.3247e-03],
         [-1.1287e-02, -1.9644e-03, -1.1573e-02,  ...,  1.4908e-02,
           1.8741e-02, -7.3140e-03],
         ...,
         [ 1.0013e-03, -1.8523e-03,  6.4189e-03,  ..., -7.2757e-03,
           1.7281e-03,  2.8407e-03],
         [-3.6395e-03,  8.9983e-03,  1.3761e-02,  ..., -3.5218e-03,
           3.5612e-04,  7.9451e-03],
         [-1.8650e-02,  7.7867e-03,  1.8862e-02,  ..., -1.9309e-02,
           2.0698e-02,  9.4798e-03]]], device='cuda:0',
       grad_fn=<EmbeddingBackward>)

In [26]:
model.position_embeddings.weight

Parameter containing:
tensor([[ 1.7505e-02, -2.5631e-02, -3.6642e-02,  ...,  3.3437e-05,
          6.8312e-04,  1.5441e-02],
        [ 7.7580e-03,  2.2613e-03, -1.9444e-02,  ...,  2.8910e-02,
          2.9753e-02, -5.3247e-03],
        [-1.1287e-02, -1.9644e-03, -1.1573e-02,  ...,  1.4908e-02,
          1.8741e-02, -7.3140e-03],
        ...,
        [ 1.7418e-02,  3.4903e-03, -9.5621e-03,  ...,  2.9599e-03,
          4.3435e-04, -2.6949e-02],
        [ 2.1687e-02, -6.0216e-03,  1.4736e-02,  ..., -5.6118e-03,
         -1.2590e-02, -2.8085e-02],
        [ 2.6413e-03, -2.3298e-02,  5.4922e-03,  ...,  1.7537e-02,
          2.7550e-02, -7.7656e-02]], device='cuda:0', requires_grad=True)

In [27]:
model.bert.distilbert.embeddings.position_embeddings.weight

Parameter containing:
tensor([[ 1.7505e-02, -2.5631e-02, -3.6642e-02,  ...,  3.3437e-05,
          6.8312e-04,  1.5441e-02],
        [ 7.7580e-03,  2.2613e-03, -1.9444e-02,  ...,  2.8910e-02,
          2.9753e-02, -5.3247e-03],
        [-1.1287e-02, -1.9644e-03, -1.1573e-02,  ...,  1.4908e-02,
          1.8741e-02, -7.3140e-03],
        ...,
        [ 1.7418e-02,  3.4903e-03, -9.5621e-03,  ...,  2.9599e-03,
          4.3435e-04, -2.6949e-02],
        [ 2.1687e-02, -6.0216e-03,  1.4736e-02,  ..., -5.6118e-03,
         -1.2590e-02, -2.8085e-02],
        [ 2.6413e-03, -2.3298e-02,  5.4922e-03,  ...,  1.7537e-02,
          2.7550e-02, -7.7656e-02]], device='cuda:0', requires_grad=True)

In [28]:
model.position_ids

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
         54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
         72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
         90, 91, 92, 93, 94, 95]], device='cuda:0')

In [29]:
word = ((out[0, :8].unsqueeze(1)-model.bert.distilbert.embeddings.word_embeddings.weight.data)**2).mean(-1).argmin(-1)

In [30]:
GetToks(Torch2Py(word))

' purpose article two [MASK] of scientific investigation and'

In [31]:
pos_emb = inp_emb_tok_pos - inp_emb_tok_no_pos